Credits: YOLO repo https://github.com/ultralytics/yolov3

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#YOLO on Video 1

In [25]:
!git clone https://github.com/ultralytics/yolov3

fatal: destination path 'yolov3' already exists and is not an empty directory.


In [26]:
%cd yolov3
import time
import glob
import torch
import os

import argparse
from sys import platform

from models import *
from utils.datasets import *
from utils.utils import *

from IPython.display import HTML
from base64 import b64encode

parser = argparse.ArgumentParser()
parser.add_argument('--cfg', type=str, default='cfg/yolov3-spp.cfg', help='*.cfg path')
parser.add_argument('--names', type=str, default='data/coco.names', help='*.names path')
parser.add_argument('--weights', type=str, default='weights/yolov3-spp-ultralytics.pt', help='weights path')

parser.add_argument('--img-size', type=int, default=416, help='inference size (pixels)')
parser.add_argument('--conf-thres', type=float, default=0.3, help='object confidence threshold')
parser.add_argument('--iou-thres', type=float, default=0.6, help='IOU threshold for NMS')


parser.add_argument('--device', default='', help='device id (i.e. 0 or 0,1) or cpu')


parser.add_argument('--classes', nargs='+', type=int, help='filter by class')
parser.add_argument('--agnostic-nms', action='store_true', help='class-agnostic NMS')
opt = parser.parse_args(args = [])

weights = opt.weights
img_size =  opt.img_size
device = torch_utils.select_device(device='cpu' if ONNX_EXPORT else opt.device)

model = Darknet(opt.cfg, img_size)

attempt_download(weights)
if weights.endswith('.pt'):
    model.load_state_dict(torch.load(weights, map_location=device)['model'])
else:
    load_darknet_weights(model, weights)

model.to(device).eval();

names = load_classes(opt.names)
colors = [[random.randint(0, 255) for _ in range(3)] for _ in range(len(names))]

%cd .. 

def predict_one_video(path_video, output_dir = 'output'): 

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    cap  = cv2.VideoCapture(path_video)
    _, img0 = cap.read()

    save_path = os.path.join(output_dir, os.path.split(path_video)[-1]) 
    fps = cap.get(cv2.CAP_PROP_FPS)
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    vid_writer = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*'MP4V'), fps, (w, h))

    while img0 is not None: 

        img = letterbox(img0, new_shape=opt.img_size)[0]

        img = img[:, :, ::-1].transpose(2, 0, 1)
        img = np.ascontiguousarray(img)

        img = torch.from_numpy(img).to(device)
        img = img.float()
        img /= 255.0
        if img.ndimension() == 3:
            img = img.unsqueeze(0)

        pred = model(img)[0]

        pred = non_max_suppression(pred, opt.conf_thres, opt.iou_thres, classes=opt.classes, agnostic=opt.agnostic_nms)

        for i, det in enumerate(pred):
            im0 = img0

            if det is not None and len(det):
                det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()

                # Write results
                for *xyxy, conf, cls in det:
                    label = '%s %.2f' % (names[int(cls)], conf)
                    plot_one_box(xyxy, im0, label=label, color=colors[int(cls)])

        vid_writer.write(im0)
        _, img0 = cap.read()

    vid_writer.release()

    return save_path


/content/yolov3
Using CUDA device0 _CudaDeviceProperties(name='Tesla T4', total_memory=15079MB)

Model Summary: 225 layers, 6.29987e+07 parameters, 6.29987e+07 gradients
/content


In [27]:
!git clone https://github.com/vindruid/yolov3-in-colab.git
!cp -r "yolov3-in-colab"/input_video/* ./input_video/

fatal: destination path 'yolov3-in-colab' already exists and is not an empty directory.


In [28]:
!mkdir -p input_video
!mkdir -p output_compressed

In [33]:
path_video = os.path.join("input_video","thevideotoworkon.mp4")
save_path = predict_one_video(path_video)

mp4 = open(path_video,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

Output hidden; open in https://colab.research.google.com to view.

In [34]:

compressed_path = os.path.join("output_compressed", os.path.split(save_path)[-1])
os.system(f"ffmpeg -i {save_path} -vcodec libx264 {compressed_path}")

mp4 = open(compressed_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

Output hidden; open in https://colab.research.google.com to view.